<a href="https://colab.research.google.com/github/kmk4444/-Calculating-potential-customer-revenue-using-rule-based-classification/blob/main/Haber_cekme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
mmöm